In [1]:
# Import necessary modules and functions
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from src.neural_nets import create_network_1, create_network_2, create_network_3, parameter_tuning_nn, median_and_iqr_nn
from src.knn_regression import parameter_tuning_knn, median_and_iqr_knn, generate_neighbors
from data.data_generator import get_data, preprocess
from data.regression_functions import m1, m2, m3, m4, m5, m6, m7, m8

In [ ]:
# Define a list of regression functions
regression_functions = [m1, m2, m3, m4, m5, m6, m7, m8]

In [ ]:
# Define a list of neural networks
neural_networks = [create_network_1, create_network_2, create_network_3]


# Define empty dataframe to store results
results_nn = pd.DataFrame(
    columns=["regression_function", "network", "units", "mse", "iqr"]
)

# Loop through each regression function
for i, regression_func in enumerate(regression_functions):
    print(f"Regression Function {i+1}: {regression_func.__name__}")
    input_dim = regression_func.expected_dim
    x, y = get_data(regression_func, x_dim=input_dim, num_samples=1000, sigma=0.05)
    x_test, y_test = get_data(
        regression_func, x_dim=input_dim, num_samples=10**5, sigma=0.05
    )
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

    # Preprocess data
    train_data = preprocess(x_train, y_train, training=True)
    test_data = preprocess(x_test, y_test, training=False)
    val_data = preprocess(x_val, y_val, training=False)

    # Tune networks
    units = [3, 6, 9, 12, 15]
    for j, network in enumerate(neural_networks):
        best_params = parameter_tuning_nn(
            network,
            units=units,
            train_data=train_data,
            test_data=val_data,
            input_dim=input_dim,
        )

        # Calculate regression for best units
        best_units = best_params["best_config"]
        mse, iqr = median_and_iqr_nn(
            network, train_data, test_data, input_dim=input_dim, units=best_units
        )

        # Append the results to the dataframe
        df_nn = pd.DataFrame(
            {
                "regression_function": [regression_func.__name__],
                "network": [network.__name__],
                "units": [best_units],
                "mse": [mse],
                "iqr": [iqr],
            }
        )
        results_nn = pd.concat([results_nn, df_nn], ignore_index=True)

        print(f"Best Units: {best_units}")
        print(f"MSE: {mse}")
        print(f"IQR: {iqr}")

    print("--------------------")

In [ ]:
# Define the file path
file_path = "../results/nn_performance.csv"

# Save the results to a csv file
results_nn.to_csv(file_path, index=False)

In [ ]:
# Define empty dataframe to store results
results_knn = pd.DataFrame(columns=["regression_function", "units", "mse", "iqr"])

# Loop through each regression function
for i, regression_func in enumerate(regression_functions):
    print(f"Regression Function {i+1}: {regression_func.__name__}")
    input_dim = regression_func.expected_dim
    x, y = get_data(regression_func, x_dim=input_dim, num_samples=1000, sigma=0.05)
    x_test, y_test = get_data(
        regression_func, x_dim=input_dim, num_samples=10**5, sigma=0.05
    )
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

    # Preprocess data
    train_data = (x_train, y_train)
    test_data = (x_test, y_test)
    val_data = (x_val, y_val)

    # Tune number of neighbors
    units = generate_neighbors(x_train.shape[0], 25)
    best_params = parameter_tuning_knn(
        units=units,
        train_data=train_data,
        test_data=val_data,
    )

    # Calculate regression for the best number of neighbors
    best_units = best_params["best_config"]
    mse, iqr = median_and_iqr_knn(
        train_data,
        test_data,
        units=best_units,
    )

    # Append the results to the dataframe
    df_knn = pd.DataFrame(
        {
            "regression_function": [regression_func.__name__],
            "units": [best_units],
            "mse": [mse],
            "iqr": [iqr],
        }
    )
    results_knn = pd.concat([results_knn, df_knn], ignore_index=True)

    print(f"Best Units: {best_units}")
    print(f"MSE: {mse}")
    print(f"IQR: {iqr}")

    print("--------------------")

In [ ]:
# Define the file path
file_path = "../results/knn_performance.csv"

# Save the results to a csv file
results_nn.to_csv(file_path, index=False)